In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

#모델링
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error

from pycaret.classification import *

In [27]:
train=pd.read_csv('../train.csv', index_col=0)
test=pd.read_csv('../test.csv', index_col=0)
submission=pd.read_csv('../sample_submission.csv', index_col=0)  
print(train.shape)
print(test.shape)
print(submission.shape)

(15000, 69)
(35452, 68)
(35452, 1)


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [29]:
#labelencoding
label = LabelEncoder()

train['gender_code'] = label.fit_transform(train['gender'])
train['religion_code'] = label.fit_transform(train['religion'])

test['gender_code'] = label.fit_transform(test['gender'])
test['religion_code'] = label.fit_transform(test['religion'])

drop2 = ['country','gender','religion']
train = train.drop(drop2, axis = 1)
test = test.drop(drop2, axis = 1)


In [5]:
#pd_df = pd.get_dummies(train[['education','urban','engnat','age','hand','orientation','voted','married','familysize','gender','religion']])

#train = pd.concat([train,pd_df], axis=1)

In [30]:
#이상치를 NaN으로 변경 
train.mask(train.sub(train.mean()).div(train.std()).abs().gt(2))
test.mask(test.sub(test.mean()).div(test.std()).abs().gt(2))

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,orientation,voted,married,familysize,ASD,gender_code,religion_code
index,,,,,,,,,,,,,,,,,,,,,
0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,...,2.0,19.0,1.0,4.0,1.0,1.0,3.0,2.0,1.0,3.0
1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,...,1.0,33.0,1.0,NaN,2.0,1.0,5.0,2.0,0.0,0.0
2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,...,2.0,13.0,1.0,NaN,2.0,1.0,3.0,2.0,1.0,3.0
3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,...,1.0,28.0,1.0,2.0,2.0,1.0,3.0,2.0,0.0,1.0
4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,2.0,15.0,1.0,NaN,2.0,1.0,2.0,2.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,...,2.0,16.0,NaN,4.0,2.0,1.0,3.0,2.0,1.0,9.0
35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,...,1.0,16.0,NaN,NaN,2.0,1.0,2.0,2.0,2.0,1.0
35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,1.0,31.0,1.0,NaN,1.0,1.0,2.0,2.0,0.0,0.0


In [31]:
#missing 확인
print('train : ',train.isnull().sum())
print('test : ',test.isnull().sum())

train :  Q1                41
Q2                69
Q3                50
Q4                71
Q5                38
                ... 
familysize       319
ASD               89
nerdiness          0
gender_code        0
religion_code      0
Length: 68, dtype: int64
test :  Q1               119
Q2               147
Q3               130
Q4               125
Q5                90
                ... 
married          180
familysize       703
ASD              237
gender_code        0
religion_code      0
Length: 67, dtype: int64


In [ ]:
#결측치 보간 
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
train = pd.DataFrame(imp_mean.fit_transform(train), columns=train.columns)
test = pd.DataFrame(imp_mean.fit_transform(test), columns=test.columns)
test

In [5]:
#train.interpolate(method='values',inplace=True)
#test.interpolate(method='values', inplace=True)

In [33]:
#민혁님이 짜주신 코드 
#상관계수가 0.1 이하면 drop
idx=[]
del_col=[]
a = train.corr()['nerdiness']
for i, value in enumerate(train.corr()['nerdiness']):
    if abs(value) < 0.1:
        idx.append(i)
        del_col.append(train.columns[i])

print(a[del_col])

train.drop(del_col, axis=1,inplace=True)
test.drop(del_col, axis=1,inplace=True)

introelapse      0.008319
testelapse       0.005550
surveyelapse     0.011953
TIPI2            0.023532
TIPI3            0.034750
TIPI4            0.043227
TIPI5            0.032999
TIPI6            0.090594
TIPI7            0.035029
TIPI8            0.033865
TIPI9           -0.019097
TIPI10          -0.095197
VCL1             0.035160
VCL2             0.073068
VCL4             0.044750
VCL5             0.051207
VCL6             0.033286
VCL7             0.031422
VCL8             0.060814
VCL9             0.016625
VCL10            0.026029
VCL11            0.056328
VCL12            0.067186
VCL14            0.087772
VCL15            0.039101
VCL16            0.023411
education       -0.028579
urban           -0.032702
engnat          -0.055815
age              0.005443
hand             0.013016
orientation      0.067163
voted           -0.013146
married         -0.025503
familysize      -0.011022
ASD             -0.056458
gender_code      0.014525
religion_code   -0.019083
Name: nerdin

In [34]:
clf = setup(data = train, target = 'nerdiness')

,Description,Value
0,session_id,8839
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 30)"
5,Missing Values,False
6,Numeric Features,27
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


In [10]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7705,0.8599,0.8278,0.7731,0.7994,0.5319,0.5337,0.1650
rf,Random Forest Classifier,0.7688,0.8558,0.8250,0.7725,0.7978,0.5287,0.5304,0.1730
lightgbm,Light Gradient Boosting Machine,0.7513,0.8222,0.8062,0.7589,0.7818,0.4934,0.4947,0.0540
gbc,Gradient Boosting Classifier,0.7341,0.8062,0.7966,0.7415,0.7679,0.4575,0.4593,0.3310
ada,Ada Boost Classifier,0.7241,0.7953,0.7805,0.7361,0.7576,0.4380,0.4391,0.0850
lda,Linear Discriminant Analysis,0.7238,0.7948,0.7976,0.7284,0.7614,0.4351,0.4378,0.0670
lr,Logistic Regression,0.7043,0.7672,0.7905,0.7082,0.7470,0.3937,0.3973,0.5120
qda,Quadratic Discriminant Analysis,0.4886,0.7203,0.0957,0.8229,0.1601,0.0635,0.1351,0.0360
dt,Decision Tree Classifier,0.6998,0.6966,0.7267,0.7292,0.7278,0.3931,0.3933,0.0310
nb,Naive Bayes,0.4478,0.6938,0.0031,0.5000,0.0061,-0.0003,0.0011,0.1440


In [36]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7648,0.8482,0.8291,0.7674,0.7970,0.5183,0.5205
1,0.7743,0.8585,0.8309,0.7790,0.8041,0.5385,0.5401
2,0.7810,0.8576,0.8403,0.7828,0.8105,0.5517,0.5537
3,0.7633,0.8429,0.8207,0.7700,0.7945,0.5162,0.5176
4,0.7670,0.8496,0.8402,0.7650,0.8008,0.5217,0.5251
Mean,0.7701,0.8514,0.8322,0.7729,0.8014,0.5293,0.5314
Std,0.0066,0.0059,0.0074,0.0069,0.0056,0.0137,0.0136


In [37]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7736,0.8568,0.8314,0.7707,0.7999,0.5401,0.5422


In [38]:
# 전체 데이터에 대한 재학습
final_model = finalize_model(blended)

In [39]:
# 대회용 test set에 대한 예측 
predictions = predict_model(final_model, data = test)

In [40]:
predictions

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q23,Q24,Q25,Q26,TIPI1,VCL3,VCL13,Label,Score_0.0,Score_1.0
0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,...,5.0,5.0,4.0,4.0,2.0,1.0,1.0,0.0,0.6436,0.3564
1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,...,5.0,5.0,4.0,5.0,1.0,1.0,1.0,1.0,0.1922,0.8078
2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,...,5.0,5.0,1.0,5.0,3.0,0.0,0.0,1.0,0.1156,0.8844
3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,...,4.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,0.3529,0.6471
4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,5.0,5.0,3.0,3.0,3.0,0.0,0.0,1.0,0.1744,0.8256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,...,4.0,4.0,5.0,3.0,3.0,0.0,0.0,1.0,0.1711,0.8289
35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,...,4.0,4.0,3.0,5.0,3.0,0.0,1.0,1.0,0.2601,0.7399
35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,4.0,5.0,1.0,1.0,1.0,1.0,0.0271,0.9729
35450,5.0,5.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,...,5.0,5.0,5.0,5.0,3.0,0.0,1.0,0.0,0.8026,0.1974


In [41]:
submission['nerdiness'] = predictions['Score']

In [42]:
#submission.to_csv('../simpleimpute_Score_corr_sy.csv')